# Chapter 23: Dynamic Programming

> *"Those who cannot remember the past are condemned to repeat it."* — George Santayana (adapted for DP)

---

## 23.1 Introduction to Dynamic Programming

**Dynamic Programming (DP)** is a powerful technique for solving problems that exhibit **overlapping subproblems** and **optimal substructure**. Instead of solving the same subproblem repeatedly, DP solves each subproblem once and stores its result, typically in a table (array or dictionary).

### 23.1.1 When to Use Dynamic Programming

DP is applicable when a problem has:

1. **Optimal Substructure:** An optimal solution to the problem contains optimal solutions to subproblems.
2. **Overlapping Subproblems:** The same subproblems are solved multiple times (distinct from divide-and-conquer, where subproblems are independent).

### 23.1.2 Why Dynamic Programming Matters

```
┌─────────────────────────────────────────────────────────────────────┐
│                    IMPORTANCE OF DYNAMIC PROGRAMMING                  │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  1. OPTIMIZATION: Solves many optimization problems that would      │
│     be intractable with brute force (exponential time).             │
│                                                                      │
│  2. WIDE APPLICABILITY: Used in bioinformatics (sequence alignment),│
│     operations research (inventory, scheduling), economics,         │
│     computer graphics (path finding), and more.                     │
│                                                                      │
│  3. FOUNDATION FOR ADVANCED TOPICS: Reinforcement learning,         │
│     control theory, and algorithm design rely on DP principles.     │
│                                                                      │
│  4. INTERVIEW FAVORITE: DP problems are common in technical         │
│     interviews at top tech companies.                               │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---

## 23.2 Key Concepts: Memoization vs Tabulation

Two main approaches to implement DP:

- **Memoization (Top-Down):** Recursive solution with caching. We write the recursive solution and store results in a dictionary or array.
- **Tabulation (Bottom-Up):** Iterative solution, filling a table from base cases up to the desired result.

### 23.2.1 Example: Fibonacci Numbers

**Recursive (without DP) – exponential time:**

```python
def fib_rec(n):
    if n <= 1:
        return n
    return fib_rec(n-1) + fib_rec(n-2)
```

**Memoization (Top-Down):**

```python
def fib_memo(n, memo={}):
    if n in memo:
        return memo[n]
    if n <= 1:
        return n
    memo[n] = fib_memo(n-1, memo) + fib_memo(n-2, memo)
    return memo[n]
```

**Tabulation (Bottom-Up):**

```python
def fib_tab(n):
    if n <= 1:
        return n
    dp = [0] * (n + 1)
    dp[1] = 1
    for i in range(2, n + 1):
        dp[i] = dp[i-1] + dp[i-2]
    return dp[n]
```

**Space-optimized tabulation:**

```python
def fib_opt(n):
    if n <= 1:
        return n
    prev2, prev1 = 0, 1
    for _ in range(2, n + 1):
        curr = prev1 + prev2
        prev2, prev1 = prev1, curr
    return prev1
```

**Time Complexity:** O(n) for all DP versions.  
**Space:** O(n) for memoization/tabulation, O(1) for optimized.

---

## 23.3 Classic DP Patterns

### 23.3.1 Linear DP (1D)

Problems where state is a single integer (often index).

**Example: Climbing Stairs (LeetCode 70)**

You are climbing a staircase. It takes n steps to reach the top. Each time you can either climb 1 or 2 steps. In how many distinct ways can you climb to the top?

```python
def climb_stairs(n):
    if n <= 2:
        return n
    dp = [0] * (n + 1)
    dp[1] = 1
    dp[2] = 2
    for i in range(3, n + 1):
        dp[i] = dp[i-1] + dp[i-2]
    return dp[n]
```

**State:** dp[i] = number of ways to reach step i.

**Recurrence:** dp[i] = dp[i-1] + dp[i-2]

### 23.3.2 Knapsack Family

**0/1 Knapsack:** Each item can be taken at most once.

Given weights w[i] and values v[i], maximize value subject to capacity W.

```python
def knapsack_01(weights, values, W):
    n = len(weights)
    dp = [[0] * (W + 1) for _ in range(n + 1)]
    for i in range(1, n + 1):
        for w in range(W + 1):
            if weights[i-1] <= w:
                dp[i][w] = max(dp[i-1][w], 
                               dp[i-1][w - weights[i-1]] + values[i-1])
            else:
                dp[i][w] = dp[i-1][w]
    return dp[n][W]
```

**Space-optimized (1D array):**

```python
def knapsack_01_opt(weights, values, W):
    n = len(weights)
    dp = [0] * (W + 1)
    for i in range(n):
        for w in range(W, weights[i] - 1, -1):
            dp[w] = max(dp[w], dp[w - weights[i]] + values[i])
    return dp[W]
```

**Time:** O(nW)  
**Space:** O(W)

**Unbounded Knapsack:** Items can be taken unlimited times.

```python
def unbounded_knapsack(weights, values, W):
    dp = [0] * (W + 1)
    for w in range(1, W + 1):
        for i in range(len(weights)):
            if weights[i] <= w:
                dp[w] = max(dp[w], dp[w - weights[i]] + values[i])
    return dp[W]
```

**Note:** Loop order matters: for unbounded, we iterate w from 1 to W and i inner, allowing multiple uses.

### 23.3.3 Longest Common Subsequence (LCS)

Given two strings s1 and s2, find length of longest subsequence present in both.

```python
def lcs(s1, s2):
    m, n = len(s1), len(s2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if s1[i-1] == s2[j-1]:
                dp[i][j] = dp[i-1][j-1] + 1
            else:
                dp[i][j] = max(dp[i-1][j], dp[i][j-1])
    return dp[m][n]
```

**Time:** O(mn)  
**Space:** O(mn) (can be optimized to O(min(m,n)))

**Reconstructing LCS:**

```python
def reconstruct_lcs(s1, s2, dp):
    i, j = len(s1), len(s2)
    lcs_str = []
    while i > 0 and j > 0:
        if s1[i-1] == s2[j-1]:
            lcs_str.append(s1[i-1])
            i -= 1
            j -= 1
        elif dp[i-1][j] > dp[i][j-1]:
            i -= 1
        else:
            j -= 1
    return ''.join(reversed(lcs_str))
```

### 23.3.4 Edit Distance (Levenshtein Distance)

Minimum number of operations (insert, delete, replace) to convert string s1 to s2.

```python
def edit_distance(s1, s2):
    m, n = len(s1), len(s2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if s1[i-1] == s2[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = 1 + min(dp[i-1][j],    # delete
                                    dp[i][j-1],    # insert
                                    dp[i-1][j-1])  # replace
    return dp[m][n]
```

### 23.3.5 Longest Increasing Subsequence (LIS)

**O(n²) DP:**

```python
def lis_length(nums):
    n = len(nums)
    dp = [1] * n
    for i in range(n):
        for j in range(i):
            if nums[j] < nums[i]:
                dp[i] = max(dp[i], dp[j] + 1)
    return max(dp)
```

**O(n log n) using patience sorting:**

```python
import bisect

def lis_length_opt(nums):
    tails = []
    for x in nums:
        i = bisect.bisect_left(tails, x)
        if i == len(tails):
            tails.append(x)
        else:
            tails[i] = x
    return len(tails)
```

### 23.3.6 Matrix Chain Multiplication

Given matrices with dimensions, find minimum number of scalar multiplications to compute product.

```python
def matrix_chain_order(dims):
    n = len(dims) - 1  # number of matrices
    dp = [[0] * n for _ in range(n)]
    # dp[i][j] = min cost to multiply matrices i..j
    for length in range(2, n + 1):
        for i in range(n - length + 1):
            j = i + length - 1
            dp[i][j] = float('inf')
            for k in range(i, j):
                cost = dp[i][k] + dp[k+1][j] + dims[i] * dims[k+1] * dims[j+1]
                dp[i][j] = min(dp[i][j], cost)
    return dp[0][n-1]
```

**Time:** O(n³)

### 23.3.7 Palindrome Partitioning (Minimum Cuts)

Given a string, partition it into substrings that are all palindromes. Find minimum cuts needed.

```python
def min_palindrome_cuts(s):
    n = len(s)
    # is_pal[i][j] = whether s[i:j+1] is palindrome
    is_pal = [[False] * n for _ in range(n)]
    for i in range(n):
        is_pal[i][i] = True
    for length in range(2, n + 1):
        for i in range(n - length + 1):
            j = i + length - 1
            if s[i] == s[j] and (length == 2 or is_pal[i+1][j-1]):
                is_pal[i][j] = True
    
    # dp[i] = min cuts for s[0:i]
    dp = [float('inf')] * n
    for i in range(n):
        if is_pal[0][i]:
            dp[i] = 0
        else:
            for j in range(i):
                if is_pal[j+1][i]:
                    dp[i] = min(dp[i], dp[j] + 1)
    return dp[n-1]
```

---

## 23.4 State Space Reduction Techniques

Often, we can reduce the dimension of DP tables by observing that only a few previous states are needed.

- **1D from 2D:** For knapsack, we used 1D array by iterating backwards.
- **Rolling arrays:** For LCS, we only need previous row, so space O(min(m,n)).
- **Bitmask DP:** For subset problems, use integer masks to represent sets.

---

## 23.5 Tree DP

DP on trees often involves a DFS that computes values for children and combines at parent.

**Example: Diameter of Binary Tree** (already covered in Chapter 11) – a tree DP.

**Example: Maximum Path Sum (LeetCode 124)**

```python
def max_path_sum(root):
    def dfs(node):
        nonlocal max_sum
        if not node:
            return 0
        left = max(dfs(node.left), 0)
        right = max(dfs(node.right), 0)
        max_sum = max(max_sum, left + right + node.val)
        return node.val + max(left, right)
    
    max_sum = float('-inf')
    dfs(root)
    return max_sum
```

**Example: House Robber III (LeetCode 337)**

```python
def rob(root):
    def dfs(node):
        if not node:
            return (0, 0)  # (rob, not_rob)
        left = dfs(node.left)
        right = dfs(node.right)
        rob = node.val + left[1] + right[1]
        not_rob = max(left) + max(right)
        return (rob, not_rob)
    
    return max(dfs(root))
```

---

## 23.6 DP on Grids

**Example: Unique Paths (LeetCode 62)**

Robot from top-left to bottom-right, only move down or right. Count paths.

```python
def unique_paths(m, n):
    dp = [[1] * n for _ in range(m)]
    for i in range(1, m):
        for j in range(1, n):
            dp[i][j] = dp[i-1][j] + dp[i][j-1]
    return dp[m-1][n-1]
```

**Space-optimized:** O(n)

**Example: Minimum Path Sum (LeetCode 64)**

Grid with non-negative numbers, find path with minimum sum.

```python
def min_path_sum(grid):
    m, n = len(grid), len(grid[0])
    dp = [[0]*n for _ in range(m)]
    dp[0][0] = grid[0][0]
    for i in range(1, m):
        dp[i][0] = dp[i-1][0] + grid[i][0]
    for j in range(1, n):
        dp[0][j] = dp[0][j-1] + grid[0][j]
    for i in range(1, m):
        for j in range(1, n):
            dp[i][j] = min(dp[i-1][j], dp[i][j-1]) + grid[i][j]
    return dp[m-1][n-1]
```

---

## 23.7 Bitmask DP

Used when state includes a subset of items (n ≤ 20 typically). Represent subset as integer bitmask.

**Example: Traveling Salesman Problem (TSP)**

Given cities and distances, find shortest tour visiting each city exactly once.

```python
def tsp(dist):
    n = len(dist)
    dp = [[float('inf')] * n for _ in range(1 << n)]
    dp[1][0] = 0  # start at city 0
    for mask in range(1 << n):
        for u in range(n):
            if not (mask & (1 << u)):
                continue
            for v in range(n):
                if mask & (1 << v):
                    continue
                dp[mask | (1 << v)][v] = min(dp[mask | (1 << v)][v],
                                              dp[mask][u] + dist[u][v])
    full = (1 << n) - 1
    ans = min(dp[full][i] + dist[i][0] for i in range(1, n))
    return ans
```

**Complexity:** O(n²·2ⁿ)

---

## 23.8 Digit DP

Used to count numbers satisfying certain digit-related constraints within a range.

**Example: Count numbers with sum of digits = k in [0, R]**

```python
def count_numbers_with_digit_sum(R, target_sum):
    digits = list(map(int, str(R)))
    n = len(digits)
    # memo[pos][sum][tight]
    from functools import lru_cache
    @lru_cache(None)
    def dp(pos, sum_so_far, tight):
        if sum_so_far > target_sum:
            return 0
        if pos == n:
            return 1 if sum_so_far == target_sum else 0
        limit = digits[pos] if tight else 9
        total = 0
        for dig in range(limit + 1):
            total += dp(pos + 1, sum_so_far + dig, tight and dig == limit)
        return total
    return dp(0, 0, True)
```

---

## 23.9 DP Optimization Techniques

### 23.9.1 Convex Hull Trick (CHT)

For DP recurrences of form dp[i] = min/max (dp[j] + a[i]*b[j] + c[j]) + constant, where slopes are monotonic, CHT can reduce from O(n²) to O(n log n) or O(n).

**Example:** dp[i] = min(dp[j] + (x[i] - x[j])²) for j < i. This is a classic "divide and conquer optimization" or CHT after expansion.

### 23.9.2 Divide and Conquer Optimization

For DP where dp[i] = min_{j < i} (dp[j] + C(j+1, i)) and the optimal decision point is monotonic (Knuth condition), we can use D&C to reduce from O(n²) to O(n log n).

**Example:** Optimal BST, certain array partitioning problems.

### 23.9.3 Knuth Optimization

For DP of form dp[i][j] = min_{i ≤ k < j} (dp[i][k] + dp[k+1][j] + w[i][j]), if w satisfies quadrangle inequality and monotonicity, then the optimal k is monotonic, leading to O(n²) instead of O(n³). Used in matrix chain multiplication.

---

## 23.10 Summary of DP Patterns

```
┌──────────────────────┬──────────────────────────────────────────────┐
│ Pattern              │ Example Problems                             │
├──────────────────────┼──────────────────────────────────────────────┤
│ Linear DP (1D)       │ Fibonacci, Climbing Stairs, House Robber    │
│ 0/1 Knapsack         │ Partition Equal Subset Sum, Target Sum      │
│ Unbounded Knapsack   │ Coin Change, Coin Change 2                   │
│ Longest Common Subseq│ Edit Distance, Minimum ASCII Delete Sum      │
│ Longest Increasing   │ Russian Doll Envelopes, Number of LIS        │
│ Interval DP          │ Matrix Chain, Palindrome Partitioning        │
│ Tree DP              │ Diameter, Max Path Sum, House Robber III    │
│ Grid DP              │ Unique Paths, Minimum Path Sum               │
│ Bitmask DP           │ TSP, Partition into K Subsets                │
│ Digit DP             │ Count numbers with digit sum, At most N      │
│ DP on DAG            │ Longest Path in DAG, Course Schedule III    │
└──────────────────────┴──────────────────────────────────────────────┘
```

---

## 23.11 Practice Problems

### Easy
1. **Climbing Stairs** (LeetCode 70)
2. **House Robber** (LeetCode 198)
3. **Maximum Subarray** (LeetCode 53) – Kadane's algorithm (DP variant)
4. **Best Time to Buy and Sell Stock** (LeetCode 121)
5. **Range Sum Query - Immutable** (LeetCode 303)

### Medium
6. **Coin Change** (LeetCode 322)
7. **Longest Increasing Subsequence** (LeetCode 300)
8. **Unique Paths** (LeetCode 62)
9. **Minimum Path Sum** (LeetCode 64)
10. **Partition Equal Subset Sum** (LeetCode 416)
11. **Coin Change 2** (LeetCode 518)
12. **Longest Common Subsequence** (LeetCode 1143)
13. **Word Break** (LeetCode 139)
14. **Combination Sum IV** (LeetCode 377)
15. **Decode Ways** (LeetCode 91)
16. **Unique Binary Search Trees** (LeetCode 96)
17. **House Robber II** (LeetCode 213)
18. **Maximum Product Subarray** (LeetCode 152)
19. **Palindromic Substrings** (LeetCode 647)
20. **Counting Bits** (LeetCode 338)

### Hard
21. **Edit Distance** (LeetCode 72)
22. **Burst Balloons** (LeetCode 312)
23. **Regular Expression Matching** (LeetCode 10)
24. **Wildcard Matching** (LeetCode 44)
25. **Dungeon Game** (LeetCode 174)
26. **Maximum Sum of 3 Non-Overlapping Subarrays** (LeetCode 689)
27. **Best Time to Buy and Sell Stock IV** (LeetCode 188)
28. **Frog Jump** (LeetCode 403)
29. **Largest Rectangle in Histogram** (LeetCode 84) – not DP but stack
30. **Count Different Palindromic Subsequences** (LeetCode 730)

---

## 23.12 Further Reading

1. **"Introduction to Algorithms" (CLRS)** – Chapter 15 (Dynamic Programming)
2. **"Algorithm Design"** by Kleinberg & Tardos – Chapter 6 (Dynamic Programming)
3. **"The Algorithm Design Manual"** by Steven Skiena – Section 8 (Dynamic Programming)
4. **"Dynamic Programming for Coding Interviews"** by Meenakshi & Kamal Rawat
5. **"Competitive Programming"** by Halim & Halim – Chapter 3 (Dynamic Programming)
6. **Online resources:** LeetCode DP section, GeeksforGeeks, AtCoder DP contest.

---

> **Coming in Chapter 24**: **String Algorithms** – We'll dive into pattern matching, suffix trees, and advanced string processing.

---

**End of Chapter 23**